In [1]:
import pandas as pd
import numpy as np

# load data
DATA_FILE = 'data/train.csv'
TEST_FILE = 'data/test.csv'
df = pd.read_csv(DATA_FILE)
TRAIN_DATA_CNT = len(df)
test_df = pd.read_csv('data/test.csv')

print df.head()
print df.shape

y_data_df = df['SalePrice']
print y_data_df.shape
x_data_df = df.drop('SalePrice',axis=1)
all_df = x_data_df.copy()
all_data_df = all_df.append(test_df) # combine train and test for encode labeler
all_data_df = all_data_df.drop(['Alley','Utilities','PoolQC','PoolArea'],axis=1)

cut_feat_cols = []
for col in all_data_df.columns:
    null_cnt = all_data_df[col].isnull().sum()
    print null_cnt,col
    if null_cnt>100:
        cut_feat_cols.append(col)
all_data_df = all_data_df.drop(cut_feat_cols,axis=1)

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2   2008     

In [2]:
all_data_df = all_data_df.apply(lambda x:x.fillna(x.value_counts().index[0])) # fillna with most frequent item

print all_data_df.shape
print all_data_df.head()

tmp_list = []
for v,c in zip(all_data_df.values[0],all_data_df.columns):
    try:
        float(v)
    except:
        print v
        tmp_list.append(c)
print tmp_list

(2919, 67)
   Id  MSSubClass MSZoning  LotArea Street LotShape LandContour LotConfig  \
0   1          60       RL     8450   Pave      Reg         Lvl    Inside   
1   2          20       RL     9600   Pave      Reg         Lvl       FR2   
2   3          60       RL    11250   Pave      IR1         Lvl    Inside   
3   4          70       RL     9550   Pave      IR1         Lvl    Corner   
4   5          60       RL    14260   Pave      IR1         Lvl       FR2   

  LandSlope Neighborhood      ...      WoodDeckSF OpenPorchSF EnclosedPorch  \
0       Gtl      CollgCr      ...               0          61             0   
1       Gtl      Veenker      ...             298           0             0   
2       Gtl      CollgCr      ...               0          42             0   
3       Gtl      Crawfor      ...               0          35           272   
4       Gtl      NoRidge      ...             192          84             0   

  3SsnPorch  ScreenPorch  MiscVal  MoSold  YrSold S

In [3]:
# data preprocessing
# should be done by read description 
cls_cols = tmp_list 
print cls_cols
print len(cls_cols)
x_col_df = all_data_df[cls_cols] 
x_num_df = all_data_df.drop(cls_cols,axis=1)
print 'col df',x_col_df.shape
print 'num df',x_num_df.shape
# use get dummy

x_col_data_new = pd.get_dummies(x_col_df)
final_all_data = np.hstack((x_num_df.values,x_col_data_new))
print final_all_data.shape
print '-'*20
print 'trans',final_all_data[0]
print 'org',all_data_df.values[0]
print '-'*20
final_x_data = final_all_data[:TRAIN_DATA_CNT]
final_x_test_data = final_all_data[TRAIN_DATA_CNT:]
print 'train x data',final_x_data.shape
print 'test x data',final_x_test_data.shape
final_y_data = y_data_df.values
print 'train y data',final_y_data.shape

['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
33
col df (2919, 33)
num df (2919, 34)
(2919, 247)
--------------------
trans [  1.00000000e+00   6.00000000e+01   8.45000000e+03   7.00000000e+00
   5.00000000e+00   2.00300000e+03   2.00300000e+03   1.96000000e+02
   7.06000000e+02   0.00000000e+00   1.50000000e+02   8.56000000e+02
   8.56000000e+02   8.54000000e+02   0.00000000e+00   1.71000000e+03
   1.00000000e+00   0.00000000e+00   2.00000000e+00   1.00000000e+00
   3.00000000e+00   1.00000000e+00   8.00000000e+00   0.00000000e+00
   2.00000000e+00   5.48000000e+02   0.00000000e+00   6.1

In [4]:
# try random forest model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn import metrics
import time
X_train, X_test, y_train, y_test = train_test_split(final_x_data, final_y_data, test_size=0.5, random_state=2333)
print np.mean(final_y_data)
print np.var(final_y_data)

import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    y = np.clip(y,0,max(y))
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

def test_model(model):
    print '-'*20
    model.fit(X_train,y_train)
    # test
    res = model.predict(X_test)
    print 'mae', metrics.mean_absolute_error(res, y_test),'rmsle',rmsle(res,y_test),

    # test on all
    s = time.time()
    res = model.predict(final_x_data)
    print '{} data time cost {}'.format(len(final_x_data), time.time() - s),
    print 'mae', metrics.mean_absolute_error(res, final_y_data),'rmsle',rmsle(res,y_test)


model = RandomForestRegressor()
test_model(model)

for m in [LinearRegression(),Lasso(),Ridge()]:
    test_model(m)

180921.19589
6306788585.35
--------------------
mae 21258.2793151 rmsle 0.169419835587 1460 data time cost 0.0559129714966 mae 14463.4393151 rmsle 0.387198389795
--------------------
mae 19830.6480315 rmsle 0.44815441286 1460 data time cost 0.0174679756165 mae 15738.401297 rmsle 0.528801351995
--------------------
mae 19541.7634083 rmsle 0.444656107376 1460 data time cost 0.00351095199585 mae 15602.9765559 rmsle 0.527236610113
--------------------
mae 18729.5456865 rmsle 0.441706684543 1460 data time cost 0.0111122131348 mae 15325.3117685 rmsle 0.525567753631


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [9]:
print 'check gbrt'
model = GradientBoostingRegressor()
test_model(model)

model = GradientBoostingRegressor(n_estimators=50)
test_model(model)

model = GradientBoostingRegressor(n_estimators=70)
test_model(model)

model = GradientBoostingRegressor(n_estimators=50,max_depth=3,subsample=0.2,min_samples_split=2,max_features='sqrt')
test_model(model)

model = GradientBoostingRegressor(n_estimators=50,max_depth=3,subsample=0.2,min_samples_split=2,max_features='sqrt')
test_model(model)

check gbrt
--------------------
mae 17994.2524385 rmsle 0.14918663433 1460 data time cost 0.0202012062073 mae 13116.2926078 rmsle 0.399748860163
--------------------
mae 18857.1397314 rmsle 0.159273823344 1460 data time cost 0.0142691135406 mae 14855.5256982 rmsle 0.388556060687
--------------------
mae 18190.8675846 rmsle 0.151990637463 1460 data time cost 0.00611281394958 mae 13849.3065995 rmsle 0.395535456533
--------------------
mae 21895.7929848 rmsle 0.181477896665 1460 data time cost 0.0181121826172 mae 19163.7321892 rmsle 0.382692833077
--------------------
mae 20713.9413294 rmsle 0.179693308309 1460 data time cost 0.029993057251 mae 18592.2696758 rmsle 0.381337624423


In [10]:
# run on test data and save to csv
final_model = GradientBoostingRegressor(n_estimators=50,max_depth=3,subsample=0.2,learning_rate=0.015,min_samples_split=2,max_features='sqrt')
final_model.fit(X_train,y_train)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res_cut_feature_dummy_part_train_gbrt.csv',index=False)

final_model = GradientBoostingRegressor(n_estimators=50,max_depth=3,subsample=0.2,learning_rate=0.015,min_samples_split=2,max_features='sqrt')
final_model.fit(final_x_data,final_y_data)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res_cut_feature_dummy_all_train_gbrt.csv',index=False)

final_model = GradientBoostingRegressor(n_estimators=50,max_features='sqrt')
final_model.fit(final_x_data,final_y_data)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res_cut_feature_dummy_all_train_gbrt_simple.csv',index=False)


[ 159721.39435946  162131.33255224  178085.11997985 ...,  167403.57343735
  160935.21227207  195567.3279018 ]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     Id      SalePrice
0  1461  159721.394359
1  1462  162131.332552
2  1463  178085.119980
3  1464  180671.658221
4  1465  191460.127826
[ 161429.53897861  169537.09929947  180218.82748713 ...,  172526.64712066
  158083.54205958  193592.64730756]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     Id      SalePrice
0  1461  161429.538979
1  1462  169537.099299
2  1463  180218.827487
3  1464  183015.064252
4  1465  187611.978229
[ 122588.57474812  157550.92144149  188358.98575668 ...,  164809.60072125
  130670.26998144  212131.82362124]
     Id      SalePrice
0  1461  122588.574748
1  1462  157550.921441
2  1463  188358.985757
3  1464  191508.609134
4  1465  198314.175622


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
